In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import animation
import imageio
import os
from tqdm import tqdm
import cv2 
import numpy as np
from PIL import Image

In [2]:
train_dir = "/kaggle/input/blood-vessel-segmentation/train/"
msks_dir = f"{train_dir}kidney_1_dense/labels/"
imgs_dir = f"{train_dir}kidney_1_dense/images/"
slices_ids = pd.Series(os.listdir(f"{msks_dir}")).sort_values().reset_index(drop=True)
slices_ids

0       0000.tif
1       0001.tif
2       0002.tif
3       0003.tif
4       0004.tif
          ...   
2274    2274.tif
2275    2275.tif
2276    2276.tif
2277    2277.tif
2278    2278.tif
Length: 2279, dtype: object

In [ ]:
idx = 1911
x = cv2.imread(imgs_dir + slices_ids[idx], cv2.IMREAD_GRAYSCALE)
xnorm1 = x / 255
xnorm2 = (x - x.min()) / (x.max() - x.min())

fig, axs = plt.subplots(2, 1)
axs[0].hist(xnorm1.reshape(-1), 50)
axs[0].set_title("255 norm")
axs[0].set_xlim([0, 1])

axs[1].hist(xnorm2.reshape(-1), 50)
axs[1].set_title("min-max norm")
axs[0].set_xlim([0, 1])

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3)
xint16 = cv2.imread(imgs_dir + slices_ids[idx], cv2.IMREAD_UNCHANGED)

axs[0].imshow(x, cmap="gray", vmin=0, vmax=255)
axs[0].axis("off")
axs[0].set_title("int8 norm")

axs[1].imshow(xint16, cmap="gray", vmin=0, vmax=2**16 - 1)
axs[1].axis("off")
axs[1].set_title("int16 norm")

axs[2].imshow(x, cmap="gray")
axs[2].axis("off")
axs[2].set_title("255 norm")

plt.tight_layout()
plt.show()

In [4]:
def make_animation(imgs_dir, masks_dir, name):
    assert sorted(os.listdir(msks_dir))  == sorted(os.listdir(imgs_dir))
    
    slices_ids = sorted(os.listdir(msks_dir))
    imgs = np.stack([cv2.imread(imgs_dir + sid, cv2.IMREAD_UNCHANGED) for sid in slices_ids], 0)
    msks = np.stack([cv2.imread(msks_dir + sid, cv2.IMREAD_UNCHANGED) for sid in slices_ids], 0)
    pbar = tqdm(total=len(slices_ids), desc=name)

    def update_plot(idx):
        idx = idx + len(slices_ids) // 2
        pbar.update(1)
        ax[0].clear()
        ax[1].clear()
        ax[0].axis("off")
        ax[1].axis("off")

        plt.suptitle(f"Slice {idx + 1} of {len(slices_ids)}")
        img = imgs[idx]
        msk = msks[idx]
        ax[0].imshow(img, vmin=0, vmax=2**16 - 1)
        ax[1].imshow(msk, vmin=0, vmax=255)
        plt.tight_layout()

    # Create the figure and axes
    fig, ax = plt.subplots(1, 2, figsize=(15, 11))
    # Create the animation
    anim = animation.FuncAnimation(fig, update_plot, frames=len(slices_ids), interval=500)
    anim.save(f'{name}.mp4', writer='ffmpeg')
    pbar.close()

In [5]:
train_dir = "/kaggle/input/blood-vessel-segmentation/train/"


kidneys = ["kidney_1_dense", "kidney_2", "kidney_3_sparse"]

for kidney in kidneys:
    imgs_dir = f"{train_dir}{kidney}/images/"
    msks_dir = f"{train_dir}{kidney}/labels/"

    make_animation(imgs_dir, msks_dir, kidney)


kidney_1_dense:   3%|▎         | 66/2279 [00:49<27:17,  1.35it/s]

kidney_1_dense:   3%|▎         | 69/2279 [00:52<30:16,  1.22it/s]